# Claude CLI - References et Contexte

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Intermediaire  
**Duree :** 25 min  
**Prerequis :** Notebooks 01-02 completes

## Objectifs d'Apprentissage

- [ ] Utiliser les @-mentions pour referencer des fichiers
- [ ] Specifier des plages de lignes (@file:10-20)
- [ ] Referencer des repertoires entiers
- [ ] Comprendre le role de CLAUDE.md
- [ ] Analyser un mini-projet complet

---

## 1. Configuration

In [ ]:
import sys
import subprocess
import os

sys.path.insert(0, 'helpers')
from claude_cli import run_claude, verify_installation, print_response

# Repertoire des exemples
EXAMPLES_DIR = os.path.join(os.getcwd(), 'examples', 'sample_project')
print(f"Repertoire exemples: {EXAMPLES_DIR}")
print(f"Existe: {os.path.exists(EXAMPLES_DIR)}")

## 2. References de Fichiers (@fichier)

Claude Code permet de referencer des fichiers directement dans vos prompts avec la syntaxe `@fichier` :

```bash
claude -p "Explique @main.py"
```

Le contenu du fichier est automatiquement inclus dans le contexte.

**Note :** En mode CLI avec `-p`, les @-mentions fonctionnent depuis le repertoire courant.

In [ ]:
# Voir le contenu du fichier main.py
main_path = os.path.join(EXAMPLES_DIR, 'main.py')
with open(main_path, 'r') as f:
    content = f.read()
print(content)

In [ ]:
# Analyser le fichier avec Claude
# En mode script, on passe le contenu directement
with open(main_path, 'r') as f:
    main_content = f.read()

stdout, stderr, code = run_claude(
    f"Analyse ce fichier Python et resume son objectif en 3 points:\n\n{main_content}"
)
print_response(stdout, stderr, code)

## 3. Plages de Lignes (@fichier:debut-fin)

Pour referencer seulement une partie d'un fichier :

```bash
# Lignes 10 a 20
claude -p "Explique @utils.py:10-20"

# A partir de la ligne 5
claude -p "Explique @utils.py:5-"

# Jusqu'a la ligne 15
claude -p "Explique @utils.py:-15"
```

In [ ]:
# Voir utils.py
utils_path = os.path.join(EXAMPLES_DIR, 'utils.py')
with open(utils_path, 'r') as f:
    lines = f.readlines()

# Afficher avec numeros de ligne
for i, line in enumerate(lines, 1):
    print(f"{i:3}: {line}", end='')

In [ ]:
# Analyser seulement la fonction calculate_statistics (lignes 15-50 environ)
relevant_lines = ''.join(lines[14:50])  # Indices Python (0-based)

stdout, stderr, code = run_claude(
    f"Explique cette fonction et identifie d'eventuels bugs:\n\n{relevant_lines}"
)
print_response(stdout, stderr, code)

## 4. Reference de Repertoires (@dossier/)

Pour inclure tous les fichiers d'un repertoire :

```bash
claude -p "Analyse la structure de @src/"
```

Claude recevra la liste des fichiers et pourra demander leur contenu.

In [ ]:
# Lister la structure du projet exemple
def list_files(directory, prefix=""):
    """Liste recursivement les fichiers d'un repertoire."""
    items = []
    for item in sorted(os.listdir(directory)):
        path = os.path.join(directory, item)
        if os.path.isfile(path):
            items.append(f"{prefix}{item}")
        elif os.path.isdir(path) and not item.startswith('.'):
            items.append(f"{prefix}{item}/")
            items.extend(list_files(path, prefix + "  "))
    return items

print("Structure du projet exemple:")
for f in list_files(EXAMPLES_DIR):
    print(f"  {f}")

In [ ]:
# Analyser le projet complet
# Collecter tous les fichiers Python
project_content = []

for root, dirs, files in os.walk(EXAMPLES_DIR):
    for file in files:
        if file.endswith('.py'):
            filepath = os.path.join(root, file)
            relpath = os.path.relpath(filepath, EXAMPLES_DIR)
            with open(filepath, 'r') as f:
                content = f.read()
            project_content.append(f"=== {relpath} ===\n{content}")

full_project = "\n\n".join(project_content)
print(f"Taille totale: {len(full_project)} caracteres")

In [ ]:
# Demander une analyse globale
stdout, stderr, code = run_claude(
    f"""Analyse ce projet Python et donne:
1. L'architecture generale
2. Les points forts
3. Les ameliorations possibles

Projet:
{full_project}"""
)
print_response(stdout, stderr, code)

## 5. Fichier CLAUDE.md

Le fichier `CLAUDE.md` a la racine d'un projet fournit un contexte permanent a Claude :

- Description du projet
- Conventions de code
- Structure des repertoires
- Commandes utiles

Claude le lit automatiquement quand vous travaillez dans le projet.

In [ ]:
# Voir notre exemple de CLAUDE.md
claude_md_path = os.path.join(os.getcwd(), 'examples', 'CLAUDE.md')
with open(claude_md_path, 'r') as f:
    claude_md = f.read()

print(claude_md)

In [ ]:
# Generer un CLAUDE.md pour un nouveau projet
stdout, stderr, code = run_claude(
    f"""A partir de ce code, genere un fichier CLAUDE.md complet:

{full_project}

Le CLAUDE.md doit inclure:
- Description du projet
- Structure des fichiers
- Commandes principales
- Conventions de code"""
)
print_response(stdout, stderr, code)

## 6. Exercices avec le Projet Exemple

In [ ]:
# EXERCICE 1 : Analysez la fonction format_report de utils.py
# et proposez des ameliorations

# Votre code ici


In [ ]:
# EXERCICE 2 : Demandez a Claude de generer des tests supplementaires
# pour la fonction normalize_data

# Votre code ici


In [ ]:
# EXERCICE 3 : Demandez une revue de code complete du projet
# avec focus sur la securite et les performances

# Votre code ici


## 7. Resume

Dans ce notebook, nous avons appris :

- **@fichier** : Inclure le contenu d'un fichier dans le prompt
- **@fichier:L1-L2** : Inclure une plage de lignes specifique
- **@dossier/** : Referencer un repertoire complet
- **CLAUDE.md** : Fournir un contexte permanent au projet

### Prochaine etape

Dans le notebook suivant, nous decouvrirons les **agents** (Explore, Plan) et les **subagents**.

-> [04-Claude-CLI-Agents.ipynb](04-Claude-CLI-Agents.ipynb)